In [63]:
import requests
import time 
import pandas as pd


headers = {
    'authority': 'www.reddit.com',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'csv=1; edgebucket=34KtHwYYCqDjvR5O5w; reddaid=R3NYJDE33LZNADQA; __gads=ID=7e647638afde60ed:T=1591212927:S=ALNI_MYncnQghNB5_SgSHNKUzPOYXC79ZA; loid=00000000006nsqcbzj.2.1591207974184.Z0FBQUFBQmZFenU4YU5SVVNJTGU0djBoRTk5QVhnQ01jODU1NkhyN2xzUjNVV0FUWmhRdWgtWldCZWJMNEtsVjRSNFdNWTViWWswMGJaaHVWcWJZWGt6MFZucmMtNDZtdEZkQlVuYS0wMVc3dGpkcHJjUDM2bHQ1MjJiMzItZ2xQNHNTcmpIS1lwdGg; G_ENABLED_IDPS=google; reddit_session=521988276655^%^2C2020-07-18T18^%^3A28^%^3A47^%^2C1381ceac8bb41be3d189ad379f494adf8e348cc5; session=7355b99ef7366899b958f01b20969214ae2eeb4egASVSQAAAAAAAABKsj8TX0dB18TP1WWEFX2UjAdfY3NyZnRflIwoMDJhYzI3Mjc0ZWYwMmM0NjA1ZWZlOGI5OWJiZDkwMWU2MDY4NGIzOZRzh5Qu; CarrotBroad_recentclicks2=t3_hqfl77; USER=eyJwcmVmcyI6eyJnbG9iYWxUaGVtZSI6IlJFRERJVCIsImNvbGxhcHNlZFRyYXlTZWN0aW9ucyI6eyJmYXZvcml0ZXMiOmZhbHNlLCJtdWx0aXMiOmZhbHNlLCJtb2RlcmF0aW5nIjpmYWxzZSwic3Vic2NyaXB0aW9ucyI6ZmFsc2UsInByb2ZpbGVzIjpmYWxzZX0sIm5pZ2h0bW9kZSI6dHJ1ZSwicnBhbkR1RGlzbWlzc2FsVGltZSI6bnVsbCwidG9wQ29udGVudERpc21pc3NhbFRpbWUiOm51bGwsInRvcENvbnRlbnRUaW1lc0Rpc21pc3NlZCI6MH0sImxhbmd1YWdlIjoiZW4ifQ==; pc=v5; __stripe_mid=1bdc2b67-eb63-4cae-9cd8-853fff86344a; __aaxsc=1; reddit_user501_recentclicks2=t3_hqfl77^%^2Ct3_ckbv15; d2_token=3.4e437285c91b5ec1b17c790525f7a216003255efe23a7f6be494ecd63ed5bd79.eyJhY2Nlc3NUb2tlbiI6IjUyMTk4ODI3NjY1NS1lbkhWZ1gwR2xpN2Y3Mm5sdEV1QlBYZGNUNUkiLCJleHBpcmVzIjoiMjAyMC0wOC0xOFQxMDowNjowNC4wMDBaIiwibG9nZ2VkT3V0IjpmYWxzZSwic2NvcGVzIjpbIioiLCJlbWFpbCJdfQ==; mnet_session_depth=2^%^7C1597741564811; recent_srs=t5_2r8ot^%^2Ct5_2qh3k^%^2Ct5_2sdpm^%^2Ct5_2szyo^%^2Ct5_2r6z7^%^2Ct5_2sgp1^%^2Ct5_2w7mz^%^2Ct5_2vegg^%^2Ct5_3cqa1^%^2Ct5_2qhqx; session_tracker=kciherjfgajkjckcci.0.1597741802113.Z0FBQUFBQmZPNXJxcTRqVXlfZkh6RDc1c3Z2WU56QkU4ZGZHSFNiSlltQkdQQk1TaXc1ZHM3UlpFckFwbFB2Q2FmNkEtcVlUZjBNaUcxNEVjM0t5Z3pZbVpReVNMWC1uTTh1MlVTSVVCRS1GUlJEOVdfVVBqSEZmUFM4am9yOHljMWhBR09xQ2J3QjE',
}
def get_mods(subreddit):
    response          = requests.get(f'https://www.reddit.com/r/{subreddit}/about/moderators/.json', headers=headers)
    json_response     = response.json()
    mods              = [{'subreddit':subreddit,'mod':item['name']} for item in json_response['data']['children'] if item['name']!='AutoModerator']
    return mods



In [59]:
# after can be passed as part of the url to get the next 25 data.
def get_mod_activity(mod):
    
    activity_list = []
    after = None
    
    for _ in range(20):
        print('running')

        if after == None:
            url = f'https://www.reddit.com/user/{mod}.json'
            response = requests.get(url,headers=headers)
        else:
            para = {}
            para['after'] = after
            url = f'https://www.reddit.com/user/{mod}.json?'
            response = requests.get(url,params=para,headers=headers)
        after = response.json()['data']['after']

        mod_activity = []
        for item in response.json()['data']['children']:
            utc_time = int(item['data']['created_utc'])
            data_dict = {'mod':mod,
               'activity_utc':utc_time,
               # from the utc_time we can extract the date and hour:
               'activity_date' : time.strftime('%Y-%m-%d',time.gmtime(utc_time)),
               'activity_hour' : time.strftime('%H',time.gmtime(utc_time))
            }
            mod_activity.append(data_dict)




        activity_list.extend(mod_activity)

    return activity_list

In [52]:
mods = get_mods('learnpython')

In [53]:
mods

[{'subreddit': 'learnpython', 'mod': 'wub_wub'},
 {'subreddit': 'learnpython', 'mod': 'xiongchiamiov'},
 {'subreddit': 'learnpython', 'mod': 'novel_yet_trivial'},
 {'subreddit': 'learnpython', 'mod': 'xelf'}]

In [60]:
all_activity = []

for item in mods:
    try:
        mod = item['mod']
        print(mod)
        mod_activity = get_mod_activity(mod)
        print(len(mod_activity))
        all_activity.extend(mod_activity)
    except:
        continue
        
#  try and incase there arent 500 of them, stop at that and continue
# to the next moderator.

wub_wub
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
500
xiongchiamiov
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
500
novel_yet_trivial
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
500
xelf
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
500


In [61]:
len(all_activity)

2000

In [62]:
all_activity[0]

{'mod': 'wub_wub',
 'activity_utc': 1597391540,
 'activity_date': '2020-08-14',
 'activity_hour': '07'}

In [64]:
df = pd.DataFrame(all_activity)
df

,mod,activity_utc,activity_date,activity_hour
0,wub_wub,1597391540,2020-08-14,07
1,wub_wub,1597340302,2020-08-13,17
2,wub_wub,1597334890,2020-08-13,16
3,wub_wub,1597334564,2020-08-13,16
4,wub_wub,1597296020,2020-08-13,05
...,...,...,...,...
1995,xelf,1593662974,2020-07-02,04
1996,xelf,1593661781,2020-07-02,03
1997,xelf,1593661584,2020-07-02,03
1998,xelf,1593661307,2020-07-02,03


In [65]:
df.to_csv('output1.csv',index=0)